In [1]:
!pip install transformers==4.40.1 accelerate==0.30.0 bitsandbytes==0.43.1 datasets==2.19.0 vllm==0.4.1 openai==1.25.1 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 1.6 MB/s eta 0:00:00
   ━━━

# 8.4절 실습: LLM 서빙 프레임워크

## 예제 8.1. 실습에 사용할 데이터셋 준비

In [1]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  dataset.loc[idx, 'prompt'] = prompt
'''
앞에 6장에서 했던거 그대로 데이터 가져와서 프롬프트 만들어주기
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


'\n앞에 6장에서 했던거 그대로 데이터 가져와서 프롬프트 만들어주기\n'

## 예제 8.2. 모델과 토크나이저를 불러와 추론 파이프라인 준비

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "shangrilar/yi-ko-6b-text2sql"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)#bnb_4bit_compute_dtype이 인자는 앞에서 배운 비츠엔 바이츠를 이용해 4비트 계산을 한다는것을 의미
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)#파이프라인 이용해서 하나의 구조를 만들어서 사용

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 예제 8.3. 배치 크기에 따른 추론 시간 확인

In [4]:
import time
for batch_size in [1, 2, 4, 8, 16, 32]: #배치 사이즈를 키워가며 추론 시간을 비교한다.
  start_time = time.time()
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

'''
결과로 보이듯이 배치가 커질수록 추론 시간이 줄어든다.
'''

1: 285.0803484916687
2: 353.18299674987793
4: 240.8259084224701
8: 173.33692598342896
16: 131.57456994056702
32: 113.54237532615662


## 예제 8.4. vLLM 모델 불러오기

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [1]:
import torch
from vllm import LLM, SamplingParams

model_id = "shangrilar/yi-ko-6b-text2sql"
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)#동시에 추론하는 데이터 수를 조정

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 11-14 06:49:28 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='shangrilar/yi-ko-6b-text2sql', speculative_config=None, tokenizer='shangrilar/yi-ko-6b-text2sql', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 11-14 06:49:28 utils.py:608] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 11-14 06:49:31 selector.py:65] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 11-14 06:49:31 selector.py:33] Using XFormers backend.
INFO 11-14 06:49:34 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 11-14 06:50:29 model_runner.py:173] Loading model weights took 11.5127 GB
INFO 11-14 06:50:31 gpu_executor.py:119] # GPU blocks: 75, # CPU blocks: 4096
INFO 11-14 06:50:35 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-14 06:50:35 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to d

## 예제 8.5. vLLM을 활용한 오프라인 추론 시간 측정

In [4]:
import time

for max_num_seqs in [1, 2, 4, 8, 16, 32]: #똑같이 배치 사이즈를 키워가며 실행
  start_time = time.time()
  llm.llm_engine.scheduler_config.max_num_seqs = max_num_seqs
  sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128) # 안에 인자들은 샘플링 공부할 때 있던 top k 개념인듯 top_p 보다 낮은 확률에 있는 확률들만 사용하고 온도 파라미터로 조정하기
  outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)
  print(f'{max_num_seqs}: {time.time() - start_time}')

Processed prompts: 100%|██████████| 112/112 [04:00<00:00,  2.15s/it]


1: 240.60184168815613


Processed prompts: 100%|██████████| 112/112 [02:07<00:00,  1.14s/it]


2: 127.82846355438232


Processed prompts: 100%|██████████| 112/112 [01:27<00:00,  1.28it/s]


...

8: 83.8617856502533


Processed prompts: 100%|██████████| 112/112 [01:22<00:00,  1.35it/s]


16: 82.91830229759216


Processed prompts: 100%|██████████| 112/112 [01:27<00:00,  1.28it/s]

32: 87.68514895439148


## 예제 8.6. 온라인 서빙을 위한 vLLM API 서버 실행

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [7]:
!python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 1024 #모델을 다운로드하고 api 서버에서 실행한다.
# 이 셀의 문제는 코랩에서 api 서버를 계속해서 실행하므로 새로운 셀을 실행해 서버에 요청을 전달할수가없다.

INFO 11-14 07:06:26 api_server.py:151] vLLM API server version 0.4.1
INFO 11-14 07:06:26 api_server.py:152] args: Namespace(host='127.0.0.1', port=8888, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, served_model_name=None, lora_modules=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], model='shangrilar/yi-ko-6b-text2sql', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, max_model_len=1024, guided_decoding_backend='outlines', worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=1, max_parallel_loading_workers=None, ray_workers_use_nsight=False, block_size=16, enable_prefix_ca

## 예제 8.7. 백그라운드에서 vLLM API 서버 실행하기

In [2]:
!nohup python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 512 &
#따라서 api 서버를 백그라운드에서 실행해야한다. 세션 초기화를 하고 위에거 실행하지말고 이거를 실행시킨다음 아래것들을 실행한다.

nohup: appending output to 'nohup.out'


## 예제 8.8. API 서버 실행 확인

In [3]:
!curl http://localhost:8888/v1/models #서버가 실행되면 답이 온다. v1/models 경로는 vllm에서 제공하는 모델 정보 제공 경로이다.

{"object":"list","data":[{"id":"shangrilar/yi-ko-6b-text2sql","object":"model","created":1731568049,"owned_by":"vllm","root":"shangrilar/yi-ko-6b-text2sql","parent":null,"permission":[{"id":"modelperm-8ffaefab76744692bc82a101d381ecfe","object":"model_permission","created":1731568049,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

## 예제 8.9. API 요청

In [4]:
import json

json_data = json.dumps(
    {"model": "shangrilar/yi-ko-6b-text2sql",
      "prompt": dataset.loc[0, "prompt"],
      "max_tokens": 128,
      "temperature": 1}
    )
#json 데이터에는 우리가 사용할 모델 이름, 입력 프롬프트, 토큰 수, 온도 파라미터와 같은 추론에 필요한 설정을 함께 전달한다.
#위와 달리 경로가 completions으로 바꼈는데 이는 vllm에서 제공하는 llm 텍스트 생성 기능이 구현된 경로이다.
!curl http://localhost:8888/v1/completions \
    -H "Content-Type: application/json" \
    -d '{json_data}'
#결과를 보면 text에 답변이 온것들 확인 할 수 있다.

{"id":"cmpl-dc9d18912b0b4cbab63e5f59fdacfba3","object":"text_completion","created":1731568049,"model":"shangrilar/yi-ko-6b-text2sql","choices":[{"index":0,"text":"SELECT reward_items, SUM(reward_experience) FROM quests GROUP BY reward_items;","logprobs":null,"finish_reason":"stop","stop_reason":null}],"usage":{"prompt_tokens":127,"total_tokens":149,"completion_tokens":22}}

## 예제 8.10. OpenAI 클라이언트를 사용한 API 요청

In [5]:
from openai import OpenAI
#vllm은 openai와 동일한 형태로 구현되어있어서 openai 클라이언트를 사용해도 요청을 전달 가능하다.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8888/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="shangrilar/yi-ko-6b-text2sql",
                                 prompt=dataset.loc[0, 'prompt'], max_tokens=128)
print("생성 결과:", completion.choices[0].text)

생성 결과: SELECT reward_items, SUM(reward_experience) AS reward_experience_total FROM quests GROUP BY reward_items;
